<a href="https://colab.research.google.com/github/Ignowsky/Payroll-PDF-Parser/blob/main/leitor_fopag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber sqlalchemy psycopg2-binary pandas

In [ ]:
import pdfplumber
import pandas as pd
import os
import re
import time

In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re
import time
from decimal import Decimal, InvalidOperation # Importa o Decimal

# --- MAPEAMENTO DE RUBRICAS (GLOBAL) ---
# (Seu mapa de rubricas original - sem alterações)
MAPEAMENTO_ORIGINAL = {
    '12': 'P_12_13_Salario_Integral', '13': 'P_13_13_Salario_Adiantamento', '19': 'P_19_Retroativo_Salarial',
    '22': 'P_22_Aviso_Previo', '28': 'P_28_Ferias_Vencidas', '29': 'P_29_Ferias_Proporcionais',
    '49': 'P_49_Aviso_Previo_Nao_Trabalhado', '50': 'P_50_Adiantamento_13_Salario', '64': 'P_64_1_3_Ferias_Rescisao',
    '150': 'P_150_Horas_Extras_50', '200': 'P_200_Horas_Extras_100', '242': 'P_242_Honorarios',
    '246': 'P_246_Diferenca_Salarial', '250': 'P_250_Reflexo_Extra_DSR', '258': 'P_258_Anuenio_Sindpd_PA',
    '263': 'P_263_Pag_Banco_Horas', '276': 'P_276_Trienio_Sindpd', '283': 'P_283_VT_Mes_Seguinte',
    '295': 'P_295_Hora_Extra_50', '314': 'P_314_Dev_Desc_Indevido', '316': 'D_316_Devolucao_Desc_Plano_Odonto', # Corrected code
    '317': 'D_317_Dev_Desc_Plano_Odonto', # Corrected code
    '340': 'P_340_Adicional_Noturno', '399': 'P_399_Banco_Horas_Pago',
    '461': 'P_461_Gratificacao_Funcao', '572': 'D_572_Dev_Desc_Plano_Odonto', # Corrected code
    '574': 'P_574_Gratificacao',
    '623': 'P_623_Gratificacao_Funcao', '695': 'P_695_Bolsa_Auxilio_Bonificacao', '700': 'P_700_Dev_Desc_INSS_Maior',
    '725': 'P_725_Dif_Plano_Medico_Dep', '763': 'P_763_Reembolso_Conselho', '766': 'P_766_Dif_Trienio',
    '800': 'P_800_Media_Horas_13', '801': 'P_801_Media_Valor_13', '802': 'P_802_Media_Fixa_13',
    '803': 'P_803_13_1_12_Indenizado', '805': 'P_805_Media_Valor_Ferias', '806': 'P_806_Media_Horas_Ferias',
    '807': 'P_807_Media_Fixa_Ferias', '808': 'P_808_Media_Valor_Abono', '809': 'P_809_Media_Horas_Abono',
    '810': 'P_810_Media_Fixa_Abono', '811': 'P_811_Ferias_1_12_Indenizado', '817': 'P_817_Media_Fer_Proporcionais',
    '820': 'P_820_Media_Ferias_Vencidas', '833': 'P_833_Media_Horas_13_Adiantado', '834': 'P_834_Media_Valor_13_Adiantado',
    '835': 'P_835_Adiocional_Fixo_13_Adiantado', '836': 'P_836_Ajuste_Inss', '846': 'P_846_Dif_Abono_Ferias',
    '854': 'P_854_Reflexo_Adicional_Noturno_DSR', '919': 'P_919_Trienio_Sinpd', '931': 'P_931_1_3_Ferias',
    '932': 'P_932_1_3_Abono_Ferias', '940': 'P_940_Diferenca_Ferias', '995': 'P_995_Salario_Familia',
    '1015': 'P_1015_Anuenio_Sindpd_PA', '8104': 'P_8104_13_Salario_Maternidade', '8112': 'P_8112_Dif_13_Ferias',
    '8126': 'P_8126_1_3_Ferias_Indenizada_Resc', '8130': 'P_8130_Estouro_Rescisao', '8158': 'P_8158_Media_Ferias_1_12_Indenizado',
    '8169': 'P_8169_1_3_Ferias_Proporcionais_Resc', '8181': 'P_8181_Dif_Media_Hora_13', '8182': 'P_8182_Dif_Media_Valor_13',
    '8184': 'P_8184_Dif_Adicional_13', '8189': 'P_8189_Dif_Media_Horas_Ferias', '8190': 'P_8190_Dif_Media_Valor_Ferias',
    '8192': 'P_8192_Dif_Media_Valor_Ferias', '8197': 'P_8197_Dif_Media_Horas_Abono_Ferias', '8200': 'P_8200_Dif_Adicional_Abono_Ferias',
    '8392': 'P_8392_13_Salario_Adiantado_Ferias', '8393': 'P_8393_Media_Horas_13_Adiantado_Ferias', '8394': 'P_8394_Media_Valor_13_Adiantado_Ferias', '8156': 'P_8156_Media_Ferias_1_12_Indenizada_Resc', '8157': 'P_8157_Media_Horas_Ferias_1_12_Indenizada_Resc', '815': 'P_815_Media_Horas_Fer_Proporcional', '816': 'P_816_Media_Valor_Fer_Proporcional',
    '8219': 'P_8219_Vantagem_13_Licenca_Maternidade', '8551': 'P_8551_Media_Horas_13_Rescisao', '8552': 'P_8552_Media_Valor_13_Rescisao',
    '9596': 'P_9596_Media_Valor_Aviso_Previo', '9597': 'P_9597_Media_Horas_Aviso_Previo', '9600': 'P_9600_Media_Valor_1_12_Indenizado',
    '9601': 'P_9601_Media_Horas_13_1_12_Indenizado','8396': 'P_8396_Vantagem_13_Adiantado', '8417': 'P_8417_Dif_1_3_Abono_Ferias', '8490': 'P_8490_Bolsa_Auxilio_Ferias_Proporcionais','8550': 'P_8550_13_Salario_Integral_Rescisao', '8553': 'P_8553_Media_13_Rescisao', '8781': 'P_8781_Salario_Empregado','8783': 'P_8783_Dias_Ferias', '8784': 'P_8784_Salario_Maternidade_Dias', '8791': 'P_8791_Dias_Afast_Dir_Integrais',
    '8797': 'P_8797_Dias_Bolsa_Estagio', '8800': 'P_8800_Dias_Abono(Ferias)', '8832': 'P_8832_Dias_Licença_Maternidade',
    '8870': 'P_8870_Dias_Afast_Doenca_Dir_Integrais', '9180': 'P_9180_Saldo_Salario_Dias', '9380': 'P_9380_Pro_Labore_Dias',
    '9591': 'P_9591_Aviso_Previo', '9592': 'P_9592_13_1_12_Indenizado', '9598': 'P_9598_Vantagem_Aviso_Indenizado',
    '9602': 'P_9602_Vantagem_13_1_12_Indenizado',
    '48': 'D_48_Vale_Transporte', '51': 'D_51_Liquido_Rescisao', '241': 'D_241_Desc_Vale_Transporte',
    '286': 'D_286_Desc_Plano_Medico_Dep', '291': 'D_291_Desc_Banco_Horas', '296': 'D_296_VT_Nao_Utilizado',
    '297': 'D_297_VA_Nao_Utilizado', '311': 'D_311_Desc_2_Via_Cartao', '325': 'D_325_Desc_Plano_Odonto',
    '331': 'D_331_Desc_Banco_Horas', '362': 'D_362_Desconto_VA_VR', '375': 'D_375_Desconto_Plano_Saude_Dep_F',
    '379': 'D_379_Desconto_Plano_Odonto_F', '394': 'D_394_Desconto_Diversos', '447': 'D_447_Desc_Plano_Odonto_Alfa_Dep',
    '449': 'D_449_Desc_Plano_Odonto_Beta', '451': 'D_451_Desc_Plano_Odonto_Alfa_Dep_F', '453': 'D_453_Desc_Plano_Odonto_Beta_F',
    '637': 'D_637_Taxa_Campanha_Sindical', '639': 'D_639_Desconto_Valor_Pago', '777': 'D_777_VT_VA_Nao_Utilizado',
    '804': 'D_804_IRRF_13', '812': 'D_812_INSS_Ferias', '821': 'D_821_Dif_Inss_Ferias',
    '825': 'D_825_Inss_13_Salario', '826': 'D_826_Inss_Sobre_Rescisao', '827': 'D_827_IRRF_13_Salario_Rescisao',
    '828': 'D_828_Irrf_Rescisao', '842': 'D_842_Multa_Estabilidade_Art_482', '843': 'D_843_Inss_Empregador',
    '856': 'D_856_Irrf_Empregador', '858': 'D_858_INSS_Autonomo', '869': 'D_869_ISS',
    '937': 'D_937_Adiantamento_Ferias', '942': 'D_942_Irrf_Ferias', '963': 'D_963_Desc_Odonto_Mais_Orto',
    '964': 'D_964_Desc_Odonto_Mais_Clarear', '965': 'D_963_Desc_Odonto_Mais_Doc', '989': 'D_989_Inss_13_Sal_Rescisao',
    '998': 'D_998_INSS', '999': 'D_999_IRRF', '1069': 'D_1069_Desc_Emprestimo_Consignado',
    '8069': 'D_8069_Faltas_Horas_Parciais', '8111': 'D_8111_Desc_Plano_Saude_Dep', '8128': 'D_8128_IRRF_Dif_Ferias',
    '8918': 'D_8918_Adiantamento_13_Media_Valor', '8919': 'D_8919_Adiantamento_13_Media_Horas', '8921': 'D_8921_Adiantamento_13_Media_Fixa',
    '9750': 'D_9750_Desc_Emprestimo_Consignado', '8214': 'D_8214_INSS_Dif_13_Salario', '8215': 'D_8215_IRRF_Dif_13_Salario',
    '8517': 'D_8517_Liquido_Rescisao_Estagiario', '8566': 'D_8566_Adiantamento_13_Salario_Rescisao'
}
proventos_map = {k: v for k, v in MAPEAMENTO_ORIGINAL.items() if v.startswith('P_')}
descontos_map = {k: v for k, v in MAPEAMENTO_ORIGINAL.items() if v.startswith('D_')}
sorted_proventos = dict(sorted(proventos_map.items(), key=lambda item: int(item[0])))
sorted_descontos = dict(sorted(descontos_map.items(), key=lambda item: int(item[0])))
MAPEAMENTO_CODIGOS = {**sorted_proventos, **sorted_descontos}
# --- FIM DO MAPEAMENTO GLOBAL ---

# --- [FUNÇÕES DE LIMPEZA ATUALIZADAS] ---

def limpar_valor(valor_str):
    """
    Converte uma string monetária para float.
    Retorna None se o valor for nulo ou inválido.
    """
    if valor_str is None:
        return None
    if isinstance(valor_str, (int, float)):
        return float(valor_str)
    if isinstance(valor_str, str):
        try:
            # Remove pontos de milhar, troca vírgula por ponto
            return float(valor_str.replace('.', '').replace(',', '.'))
        except (ValueError, TypeError):
            # Retorna None se a string não for um número válido
            return None
    # Retorna None para outros tipos (ex: objetos inesperados)
    return None

def mapear_rubrica(codigo, descricao):
    """
    Busca no mapa global a descrição e o tipo (Provento/Desconto) da rubrica.
    Retorna (codigo_limpo, nome_limpo, tipo_rubrica).
    """
    codigo_limpo = str(codigo).strip()

    if codigo_limpo in MAPEAMENTO_CODIGOS:
        nome_mapeado_completo = MAPEAMENTO_CODIGOS[codigo_limpo] # ex: 'P_12_13_Salario_Integral'
        partes = nome_mapeado_completo.split('_', 2)
        tipo_rubrica = 'Provento' if partes[0] == 'P' else 'Desconto'
        nome_limpo = partes[2]
        return codigo_limpo, nome_limpo, tipo_rubrica

    # Fallback se não estiver no mapa
    descricao_limpa = re.sub(r'[\d\s/]+$', '', descricao).strip()
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa).upper()
    nome_fallback = f"NAO_MAPEADO_{descricao_limpa}"
    # MODIFICADO: Retorna None ao invés de 'N/A'
    return codigo_limpo, nome_fallback, None


def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        # MODIFICADO: Usa 'None' como padrão
        'competencia': competencia_match.group(1).strip() if competencia_match else None,
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else None
    }

# --- [NOVA FUNÇÃO DE TRATAMENTO DE TIPOS] ---

def converter_para_decimal(valor):
    """Função auxiliar para converter valores (float ou str) para Decimal."""
    if valor is None or pd.isna(valor):
        return None
    try:
        # Converte o valor (que é float) para string e depois para Decimal
        # Isso evita problemas de precisão do float (ex: 10.1 vira 10.09999)
        return Decimal(str(valor))
    except (InvalidOperation, ValueError, TypeError):
        return None

def tratar_tipos_para_postgres(df):
    """
    Converte as colunas de um DataFrame para os tipos corretos (datetime, Decimal, string)
    ANTES de enviar ao Postgres.
    """
    if df is None or df.empty:
        return df

    print(f"\nIniciando conversão de tipos para {df.shape[1]} colunas...")

    # --- COLUNAS DE DATA (Formato DD/MM/YYYY) ---
    colunas_data = ['data_admissao', 'data_demissao']
    for col in colunas_data:
        if col in df.columns:
            print(f"Convertendo coluna de data: {col}")
            # errors='coerce' transforma datas inválidas (or None) em NaT (Not a Time)
            df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce', dayfirst=False)

    # --- COLUNA DE COMPETÊNCIA (Formato especial MM/YYYY) ---
    if 'competencia' in df.columns:
        print("Convertendo coluna de data: competencia (MM/YYYY)")
        # Adiciona '01/' para criar uma data completa e válida
        # errors='coerce' trata valores nulos (None) ou formatos ruins
        df['competencia'] = pd.to_datetime('01/' + df['competencia'].astype(str), format='%d/%m/%Y', errors='coerce')

    # --- COLUNAS MONETÁRIAS (Para NUMERIC/Decimal) ---
    colunas_monetarias = [
        'salario_contratual', 'total_proventos', 'total_descontos',
        'valor_liquido', 'base_inss', 'base_fgts', 'valor_fgts',
        'base_irrf', 'valor_rubrica'
    ]
    for col in colunas_monetarias:
        if col in df.columns:
            print(f"Convertendo coluna monetária para Decimal: {col}")
            # A função limpar_valor já retornou float.
            # Agora convertemos de float para Decimal de forma segura.
            df[col] = df[col].apply(converter_para_decimal)

    # --- COLUNAS DE TEXTO (String/Object) ---
    # Limpa espaços em branco e padroniza nulos
    # Consider columns that *should* be objects
    colunas_texto_candidatas = [col for col in df.columns if col not in colunas_data + colunas_monetarias + ['cpf', 'codigo_rubrica']]

    for col in colunas_texto_candidatas:
        if col in df.columns:
            print(f"Limpando coluna de texto candidata: {col}")
            # Attempt conversion to string and then apply strip/replace
            try:
                df[col] = df[col].astype(str).str.strip()
                # Replace 'None' string created by astype(str) for actual None values
                df[col] = df[col].replace('None', None)
                # Substitui strings vazias por None (que vira NULL)
                df[col] = df[col].replace('', None)
            except Exception as e:
                print(f"Warning: Could not process column '{col}' as string. Error: {e}")
                # If conversion or string operations fail, leave the column as is or handle differently
                pass


    # Tratamento especial para CPF (remover formatação)
    if 'cpf' in df.columns:
         print("Limpando e padronizando coluna: cpf")
         # Ensure it's string before applying replace
         df['cpf'] = df['cpf'].astype(str).str.replace(r'[^\d]', '', regex=True).replace('', None)
         df['cpf'] = df['cpf'].replace('None', None) # Replace 'None' string if any

    # Garante que codigo_rubrica seja texto
    if 'codigo_rubrica' in df.columns:
         print("Padronizando coluna: codigo_rubrica")
         # Ensure it's treated as string before replacing 'None'
         df['codigo_rubrica'] = df['codigo_rubrica'].astype(str).replace('None', None)

    print("Conversão de tipos finalizada.")
    return df

# --- [FUNÇÃO PRINCIPAL MODIFICADA] ---

def processar_pdfs_na_pasta(pasta_path):
    """
    Função principal que varre uma pasta, processa todos os PDFs e
    retorna DOIS DataFrames: um consolidado (totais) e um detalhado (rubricas).
    """
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None, None

    lista_geral_rubricas_detalhadas = []
    lista_geral_consolidados = []

    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])
                info_base = extrair_info_base(texto_completo_pdf)
                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())
                blocos_encontrados = re.finditer(r'((?:Empr|Contr)\.?\s*:\s*\d+.*?)(?=\n(?:Empr|Contr)\.?\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_processados_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(1)

                    if "CPF:" not in bloco: continue

                    posicao_bloco = bloco_match.start()
                    # MODIFICADO: Usa 'None' como padrão
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), None)

                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    vinculo_match = re.search(r'(Empr|Contr)\.?', bloco)
                    # MODIFICADO: Usa 'None' como padrão
                    dados_funcionario['vinculo'] = 'Empregado' if vinculo_match and 'Empr' in vinculo_match.group(0) else 'Contribuinte' if vinculo_match else None

                    situacao_match = re.search(r'Situação:\s*([^\n\r]+)', bloco)
                    if situacao_match:
                        situacao_str = re.split(r'\s+CPF:|\s+Adm:', situacao_match.group(1))[0].strip()
                        dados_funcionario['situacao'] = situacao_str
                    else:
                        header_chunk_match = re.search(r'(?:Empr|Contr)\.?\s*:\s*\d+.*?(?=\n|CPF:)', bloco, re.DOTALL)
                        if header_chunk_match:
                            header_chunk = header_chunk_match.group(0)
                            unlabeled_status_match = re.search(r'\s(Trabalhando|Afastado|Férias|Demitido)\s*$', header_chunk, re.IGNORECASE)
                            if unlabeled_status_match:
                                dados_funcionario['situacao'] = unlabeled_status_match.group(1)
                            else:
                                dados_funcionario['situacao'] = None # MODIFICADO
                        else:
                             dados_funcionario['situacao'] = None # MODIFICADO

                    demissao_motivo_match = re.search(r'DEMITIDO EM\s+(\d{2}/\d{2}/\d{4})\s*-\s*(.*?)(?=\n|$)', bloco, re.IGNORECASE | re.DOTALL)
                    if demissao_motivo_match:
                        dados_funcionario['data_demissao'] = demissao_motivo_match.group(1).strip()
                        dados_funcionario['motivo_demissao'] = demissao_motivo_match.group(2).strip()
                    else:
                        demissao_match_antigo = re.search(r'(?:Data Demissão|Demissão):\s*(\d{2}/\d{2}/\d{4})', bloco, re.IGNORECASE)
                        # MODIFICADO: Usa 'None' como padrão
                        dados_funcionario['data_demissao'] = demissao_match_antigo.group(1).strip() if demissao_match_antigo else None
                        dados_funcionario['motivo_demissao'] = None

                    delimitadores_nome = r'(?=\s*Situação:|\s*CPF:|\s*Adm:|\n)'
                    regex_nome = r'(?:Empr|Contr)\.?\s*:\s*\d+\s+(.*?)' + delimitadores_nome
                    nome_match = re.search(regex_nome, bloco, re.DOTALL | re.IGNORECASE)

                    if nome_match:
                        nome_capturado = nome_match.group(1).replace('\n', ' ').strip()
                        status_encontrado = dados_funcionario.get('situacao', None) # MODIFICADO
                        nome_limpo = nome_capturado
                        if status_encontrado != None and nome_limpo.lower().endswith(status_encontrado.lower()):
                            tamanho_status = len(status_encontrado)
                            nome_limpo = nome_limpo[:-tamanho_status].strip()
                        nome_limpo = re.sub(r'[^\s]+:\s*$', '', nome_limpo).strip()
                        dados_funcionario['nome_funcionario'] = nome_limpo
                    else:
                        dados_funcionario['nome_funcionario'] = None # MODIFICADO

                    cpf_match = re.search(r'CPF:\s*([\d\.\-]+)', bloco)
                    # MODIFICADO: Usa 'None' como padrão
                    dados_funcionario['cpf'] = cpf_match.group(1).strip() if cpf_match else None
                    admissao_match = re.search(r'Adm?:\s*(\d{2}/\d{2}/\d{4})', bloco)
                    dados_funcionario['data_admissao'] = admissao_match.group(1).strip() if admissao_match else None # MODIFICADO
                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)(?=\s+Salário:|\s+C\.|С\.)', bloco, re.DOTALL)
                    dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip() if cargo_match else None # MODIFICADO

                    # MODIFICADO: Salário já usa limpar_valor, que retorna float ou None
                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1)) if salario_match else None

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""

                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)

                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })
                    # MODIFICADO: Adiciona 'else' para garantir que as colunas existam (com None)
                    else:
                        dados_funcionario.update({
                            'total_proventos': None, 'total_descontos': None, 'valor_liquido': None,
                            'base_inss': None, 'base_fgts': None, 'valor_fgts': None, 'base_irrf': None
                        })

                    # 1. Salva a linha CONSOLIDADA
                    lista_geral_consolidados.append(dados_funcionario.copy())

                    # 2. Define as CHAVES para a tabela detalhada
                    chaves_funcionario_para_rubricas = {
                        'competencia': dados_funcionario.get('competencia'),
                        'tipo_calculo': dados_funcionario.get('tipo_calculo'),
                        'departamento': dados_funcionario.get('departamento'),
                        'vinculo': dados_funcionario.get('vinculo'),
                        'nome_funcionario': dados_funcionario.get('nome_funcionario'),
                        'cpf': dados_funcionario.get('cpf')
                    }

                    # 3. Processa as RUBRICAS
                    rubricas_extraidas = []
                    inicio_tabela = max(bloco.find("CPF:"), bloco.find("CPF:"))
                    fim_tabela = bloco.find("\nND:")

                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]
                        for linha in tabela_str:
                            if not re.search(r'\d', linha): continue
                            padrao_verba = r'(\d+)\s+(.*?)\s+([\d\.,]+)\s+([PD])(?=\s+\d{2,}|$)'
                            matches = re.finditer(padrao_verba, linha)
                            for match in matches:
                                valor_limpo = limpar_valor(match.group(3))
                                if valor_limpo == 0: continue
                                codigo_bruto = match.group(1)
                                descricao_bruta = match.group(2).strip()
                                descricao_bruta = re.sub(r'\s[\d\.,%]+$', '', descricao_bruta).strip()
                                (codigo_limpo, nome_mapeado, tipo_rubrica) = mapear_rubrica(codigo_bruto, descricao_bruta)
                                rubricas_extraidas.append({
                                    'codigo_rubrica': codigo_limpo,
                                    'nome_rubrica': nome_mapeado,
                                    'tipo_rubrica': tipo_rubrica,
                                    'valor_rubrica': valor_limpo
                                })

                    # 4. Adiciona as rubricas à lista DETALHADA
                    if rubricas_extraidas:
                        for rubrica in rubricas_extraidas:
                            nova_linha_longa = chaves_funcionario_para_rubricas.copy()
                            nova_linha_longa.update(rubrica)
                            lista_geral_rubricas_detalhadas.append(nova_linha_longa)
                    else:
                        # MODIFICADO: Preenche com None e 0.0 para manter tipos
                        linha_sem_rubrica = chaves_funcionario_para_rubricas.copy()
                        linha_sem_rubrica.update({
                            'codigo_rubrica': None,
                            'nome_rubrica': None,
                            'tipo_rubrica': None,
                            'valor_rubrica': 0.0 # Zero é aceitável para "sem rubrica"
                        })
                        lista_geral_rubricas_detalhadas.append(linha_sem_rubrica)

                    funcionarios_processados_no_arquivo += 1

            print(f"      - Sucesso! Foram processados {funcionarios_processados_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"    ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_consolidados and not lista_geral_rubricas_detalhadas:
        print("\nProcesso concluído, mas nenhum dado pôde ser extraído.")
        return None, None

    # --- [CRIAÇÃO E TRATAMENTO DOS DATAFRAMES FINAIS] ---

    # 1. DataFrame CONSOLIDADO (Totais)
    df_consolidado = pd.DataFrame(lista_geral_consolidados)
    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario',
        'situacao', 'data_demissao', 'motivo_demissao', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes_consol = [col for col in colunas_info_pessoal if col in df_consolidado.columns]
    if not df_consolidado.empty:
        df_consolidado = df_consolidado[colunas_presentes_consol]

    # 2. DataFrame DETALHADO (Rubricas)
    df_detalhado = pd.DataFrame(lista_geral_rubricas_detalhadas)
    colunas_chaves = [
         'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario', 'cpf'
    ]
    colunas_rubrica_longa = [
        'codigo_rubrica', 'nome_rubrica', 'tipo_rubrica', 'valor_rubrica'
    ]
    colunas_presentes_chaves = [col for col in colunas_chaves if col in df_detalhado.columns]
    colunas_presentes_rubricas = [col for col in colunas_rubrica_longa if col in df_detalhado.columns]
    ordem_final_detalhada = colunas_presentes_chaves + colunas_presentes_rubricas
    if not df_detalhado.empty:
        df_detalhado = df_detalhado[ordem_final_detalhada]

    # --- [REMOÇÃO DA FUNÇÃO 'preencher_nulos'] ---
    # Não vamos mais preencher nulos com 'N/A' ou 0.0.
    # O valor 'None' (que vira pd.NA ou pd.NaT) é o correto para o banco (NULL).

    # --- [ETAPA FINAL DE CONVERSÃO DE TIPOS] ---
    df_consolidado_tratado = tratar_tipos_para_postgres(df_consolidado)
    df_detalhado_tratado = tratar_tipos_para_postgres(df_detalhado)

    return df_detalhado_tratado, df_consolidado_tratado

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    inicio_tempo = time.perf_counter()
    # ATENÇÃO: Coloque o caminho correto para sua pasta de PDFs
    caminho_da_pasta = r'C:\Users\João Pedro\Desktop\Leitor-FOPAG\FOPAG' # Exemplo para Google Colab
    # Em ambiente local, seria algo como:
    # caminho_da_pasta = r'C:\Meus Documentos\Folha Pagamento\PDFs'

    # Os dataframes retornados agora estão com os tipos corretos
    df_detalhado_final, df_consolidado_final = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado_final is not None and not df_consolidado_final.empty:
        # Mostra os tipos de dados (dtypes) do DataFrame final
        print("\n--- Tipos de Dados Finais (Consolidado) ---")
        df_consolidado_final.info()

        nome_arquivo_saida_consol = 'BASE_FOPAG_CONSOLIDADA_TOTAIS.csv'
        # Salva em CSV (para manter seu processo original)
        df_consolidado_final.to_csv(nome_arquivo_saida_consol, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base CONSOLIDADA (Totais) foi salva em: {os.path.abspath(nome_arquivo_saida_consol)}")
    else:
        print("\nNenhum dado CONSOLIDADO foi gerado.")

    if df_detalhado_final is not None and not df_detalhado_final.empty:
        # Mostra os tipos de dados (dtypes) do DataFrame final
        print("\n--- Tipos de Dados Finais (Detalhado) ---")
        df_detalhado_final.info()

        nome_arquivo_saida_detal = 'BASE_FOPAG_DETALHADA_RUBRICAS.csv'
        # Salva em CSV
        df_detalhado_final.to_csv(nome_arquivo_saida_detal, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print(f"Sua base DETALHADA (Rubricas) foi salva em: {os.path.abspath(nome_arquivo_saida_detal)}")
    else:
        print("Nenhum dado DETALHADO de rubricas foi gerado.")

    if df_consolidado_final is None and df_detalhado_final is None:
         print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

    fim_tempo = time.perf_counter()
    duracao = (fim_tempo - inicio_tempo)
    print(f"\nTempo total de execução: {duracao:.2f} minutos") # Mudei para segundos

Encontrados 37 PDFs para processar...

---> Processando arquivo: 01.2023 ARQDIGITAL - Folha de Pagamento c.Prolabore Carol.pdf
      - Sucesso! Foram processados 40 funcionários neste arquivo.

---> Processando arquivo: 04.2023 ARQDIGITAL - Folha de Pagamento.pdf
      - Sucesso! Foram processados 41 funcionários neste arquivo.

---> Processando arquivo: 04.2024 ARQ - folha de Pagamento.pdf
      - Sucesso! Foram processados 68 funcionários neste arquivo.

---> Processando arquivo: 05.2023 ARQDIGITAL - Folha de Pagamento.pdf
      - Sucesso! Foram processados 41 funcionários neste arquivo.

---> Processando arquivo: 05.2024 ARQDIGITAL - Folha de Pagamento.pdf
      - Sucesso! Foram processados 68 funcionários neste arquivo.

---> Processando arquivo: 06.2023 ARQDIGITAL - Folha de Pagamento.pdf
      - Sucesso! Foram processados 45 funcionários neste arquivo.

---> Processando arquivo: 06.2024 ARQDIGITAL - Folha de Pagamento 2.pdf
      - Sucesso! Foram processados 69 funcionários neste

In [ ]:
df = pd.read_csv(r'C:\Users\João Pedro\Desktop\Leitor-FOPAG\BASE_FOPAG_CONSOLIDADA_TOTAIS.csv',sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2137 entries, 0 to 2136
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   competencia         2137 non-null   object 
 1   tipo_calculo        2137 non-null   object 
 2   departamento        2094 non-null   object 
 3   vinculo             2137 non-null   object 
 4   nome_funcionario    2137 non-null   object 
 5   situacao            2137 non-null   object 
 6   data_demissao       33 non-null     object 
 7   motivo_demissao     33 non-null     object 
 8   cargo               2137 non-null   object 
 9   data_admissao       2137 non-null   object 
 10  cpf                 2137 non-null   int64  
 11  salario_contratual  2137 non-null   float64
 12  total_proventos     2112 non-null   float64
 13  total_descontos     2112 non-null   float64
 14  valor_liquido       2112 non-null   float64
 15  base_inss           2112 non-null   float64
 16  base_f

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect
# Importa os tipos de dados do SQLAlchemy
from sqlalchemy.types import String, Date, Numeric
from decimal import Decimal, InvalidOperation
import io
import re

print("Iniciando processo de carga INCREMENTAL da FOPAG para o Postgres...")

# --- PASSO 1: CONFIGURAÇÃO DO BANCO ---
DB_USER = 'postgres'
DB_PASS = 'joao1502'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'postgres'
DB_SCHEMA = 'FOPAG_Long' # Schema que você quer usar

connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(connection_string)
    inspector = inspect(engine)
    print(f"Conexão com o banco '{DB_NAME}' estabelecida com sucesso.")

    # Garante que o schema exista
    with engine.begin() as conn:
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS \"{DB_SCHEMA}\""))
        print(f"Schema '{DB_SCHEMA}' verificado/criado com sucesso.")

except Exception as e:
    print(f"Erro ao conectar ou criar schema: {e}")
    exit()

# --- PASSO 2: DEFINIÇÃO EXPLÍCITA DOS SCHEMAS DAS TABELAS ---
# Define os tipos de coluna corretos para o SQL

schema_totais = {
    'competencia': Date(),
    'tipo_calculo': String(),
    'departamento': String(),
    'vinculo': String(),
    'nome_funcionario': String(),
    'situacao': String(),
    'data_demissao': Date(),
    'motivo_demissao': String(),
    'cargo': String(),
    'data_admissao': Date(),
    'cpf': String(11),
    'salario_contratual': Numeric(10, 2),
    'total_proventos': Numeric(10, 2),
    'total_descontos': Numeric(10, 2),
    'valor_liquido': Numeric(10, 2),
    'base_inss': Numeric(10, 2),
    'base_fgts': Numeric(10, 2),
    'valor_fgts': Numeric(10, 2),
    'base_irrf': Numeric(10, 2)
}

schema_rubricas = {
    'competencia': Date(),
    'tipo_calculo': String(),
    'departamento': String(),
    'vinculo': String(),
    'nome_funcionario': String(),
    'cpf': String(11),
    'codigo_rubrica': String(),
    'nome_rubrica': String(),
    'tipo_rubrica': String(),
    'valor_rubrica': Numeric(10, 2)
}

# --- PASSO 3: FUNÇÕES DE TRATAMENTO DE TIPOS (VERSÃO CORRIGIDA) ---

def clean_text(series):
    """Limpa uma série de texto (object) de forma segura."""
    if series.dtype == 'object':
        series = series.str.strip()
        series = series.str.replace(u'\xa0', '', regex=False)
        # Substitui "N/A" e strings vazias por None (que vira NULL)
        series = series.replace(['N/A', '', 'nan', 'None'], None)
    return series

def para_decimal(valor_str):
    """Converte uma string (já limpa) para Decimal."""
    if valor_str is None or pd.isna(valor_str):
        return None
    try:
        # A limpeza de \xa0 e strip já foi feita
        # Apenas removemos pontos e trocamos vírgula
        return Decimal(valor_str.replace('.', '').replace(',', '.'))
    except (InvalidOperation, ValueError, TypeError):
        return None

def tratar_tipos_dataframe_csv(df, nome_arquivo):
    """
    Função de tratamento final, ponto-a-ponto, baseada na análise dos arquivos.
    """
    print(f"Iniciando tratamento de tipos para {nome_arquivo}...")

    # --- [CORREÇÃO] COLUNAS DE DATA (TODAS AS DATAS) ---
    # Lista de todas as colunas de data que seus arquivos podem ter
    colunas_data = ['competencia', 'data_admissao', 'data_demissao']

    for col in colunas_data:
        if col in df.columns:
            # 1. Limpa a coluna de texto
            print(f"Tratando tipo de data: {col}")
            df[col] = clean_text(df[col])

            # 2. Pega o primeiro valor para checar o formato
            first_valid = df[col].dropna().iloc[0] if not df[col].dropna().empty else None

            # 3. Aplica a regra de conversão correta
            if first_valid and '/' in first_valid:
                # Lógica para formato com barra (ex: "01/2023" ou "02/01/2023")
                print(f"  -> Formato com '/' detectado. Usando regra DD/MM/YYYY ou MM/YYYY.")
                if col == 'competencia':
                    # Formato MM/YYYY
                    df[col] = pd.to_datetime('01/' + df[col], format='%d/%m/%Y', errors='coerce')
                else:
                    # Formato DD/MM/YYYY
                    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')

            elif first_valid and '-' in first_valid:
                # Lógica para formato com traço (ex: "2023-01-01")
                print(f"  -> Formato com '-' detectado. Usando parser nativo (YYYY-MM-DD).")
                df[col] = pd.to_datetime(df[col], errors='coerce') # Parser nativo

            else:
                 print(f"  -> Nenhum formato detectado ou coluna vazia. Convertendo com parser nativo.")
                 df[col] = pd.to_datetime(df[col], errors='coerce') # Parser nativo

            # 4. Converte para AAAA-MM-DD para o banco
            df[col] = df[col].dt.date

    # --- COLUNAS MONETÁRIAS ---
    colunas_monetarias = [
        'salario_contratual', 'total_proventos', 'total_descontos',
        'valor_liquido', 'base_inss', 'base_fgts', 'valor_fgts',
        'base_irrf', 'valor_rubrica'
    ]
    for col in colunas_monetarias:
        if col in df.columns:
            print(f"Tratando tipo: {col} (String -> Decimal)")
            df[col] = clean_text(df[col]) # Limpa primeiro
            df[col] = df[col].apply(para_decimal) # Converte depois

    # --- COLUNA CPF ---
    if 'cpf' in df.columns:
        print("Tratando tipo: cpf (String -> String Limpa)")
        df['cpf'] = clean_text(df['cpf'])
        df['cpf'] = df['cpf'].str.replace(r'[^\d]', '', regex=True)

    # --- COLUNAS DE TEXTO RESTANTES ---
    colunas_texto = [
        'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario',
        'situacao', 'motivo_demissao', 'cargo', 'codigo_rubrica',
        'nome_rubrica', 'tipo_rubrica'
    ]
    for col in colunas_texto:
        if col in df.columns:
            print(f"Limpando coluna de texto: {col}")
            df[col] = clean_text(df[col])

    print("Tratamento de tipos finalizado.")
    return df

# --- PASSO 4: DEFINIÇÃO DOS ARQUIVOS ---
arquivos_para_carregar = {
    'BASE_FOPAG_CONSOLIDADA_TOTAIS.csv': 'fopag_totais',
    'BASE_FOPAG_DETALHADA_RUBRICAS.csv': 'fopag_rubricas_detalhe'
}

# --- PASSO 5: LOOP DE ETL COM LÓGICA DE SCHEMA E DTYPE ---
for nome_arquivo, nome_tabela in arquivos_para_carregar.items():

    print(f"\n--- Processando arquivo: {nome_arquivo} ---")

    try:
        # --- (E) EXTRACT ---
        df = pd.read_csv(
            nome_arquivo,
            sep=';',
            na_values=['N/A', 'NaN', ''],
            dtype=str  # Força tudo a ser lido como string
        )

        # --- [VALIDAÇÃO DA LEITURA - CONFORME SOLICITADO] ---
        print(f"Arquivo lido com sucesso (como texto). {len(df)} linhas carregadas.")
        print(f"--- Amostra (head) de {nome_arquivo} (dados crus):")
        print(df.head())
        print(f"--- Info de {nome_arquivo} (dados crus):")
        df.info()
        # --- FIM DA VALIDAÇÃO ---

        # --- (T) TRANSFORM ---
        df_tratado = tratar_tipos_dataframe_csv(df.copy(), nome_arquivo)

        print(f"--- Info de {nome_arquivo} (APÓS tratamento):")
        df_tratado.info() # Mostra os tipos de dados convertidos

        dtype_map = schema_totais if nome_tabela == 'fopag_totais' else schema_rubricas

        competencias_no_df = df_tratado['competencia'].dropna().unique()

        if len(competencias_no_df) == 0:
            print(f"AVISO: Nenhuma competência válida encontrada após tratamento. Pulando carga.")
            print("Amostra da coluna 'competencia' ORIGINAL (antes da falha):")
            print(df['competencia'].head(10))
            continue

        print(f"Competências a serem carregadas: {competencias_no_df}")

        # --- (L) LOAD ---

        table_exists = inspector.has_table(nome_tabela, schema=DB_SCHEMA)

        if table_exists:
            print(f"Tabela '\"{DB_SCHEMA}\".\"{nome_tabela}\"' já existe. Executando Delete-then-Append...")

            with engine.begin() as conn:
                sql_delete = text(f"""
                    DELETE FROM "{DB_SCHEMA}"."{nome_tabela}"
                    WHERE competencia IN :competencias_list
                """)
                conn.execute(sql_delete, {"competencias_list": tuple(competencias_no_df)})
                print("DELETE executado. Iniciando INSERT (append)...")

                df_tratado.to_sql(
                    name=nome_tabela,
                    con=conn,
                    if_exists='append',
                    index=False,
                    schema=DB_SCHEMA,
                    dtype=dtype_map
                )

                print(f"SUCESSO! Dados (append) carregados na tabela '\"{DB_SCHEMA}\".\"{nome_tabela}\"'.")

        else:
            print(f"Tabela '\"{DB_SCHEMA}\".\"{nome_tabela}\"' não existe. Criando e inserindo (primeira carga)...")

            df_tratado.to_sql(
                name=nome_tabela,
                con=engine,
                if_exists='fail',
                index=False,
                schema=DB_SCHEMA,
                dtype=dtype_map # Essencial: Cria a tabela com os tipos corretos
            )

            print(f"SUCESSO! Tabela '\"{DB_SCHEMA}\".\"{nome_tabela}\"' criada com tipos corretos e dados carregados.")

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{nome_arquivo}' não encontrado.")
    except Exception as e:
        print(f"ERRO GERAL ao processar o arquivo '{nome_arquivo}': {e}")

print("\nProcesso de carga incremental finalizado.")

Iniciando processo de carga INCREMENTAL da FOPAG para o Postgres...
Conexão com o banco 'postgres' estabelecida com sucesso.
Schema 'FOPAG_Long' verificado/criado com sucesso.

--- Processando arquivo: BASE_FOPAG_CONSOLIDADA_TOTAIS.csv ---
Arquivo lido com sucesso (como texto). 2137 linhas carregadas.
--- Amostra (head) de BASE_FOPAG_CONSOLIDADA_TOTAIS.csv (dados crus):
  competencia                                 tipo_calculo  \
0  2023-01-01  Folha Mensal e Complementar Horas: 20:50:15   
1  2023-01-01  Folha Mensal e Complementar Horas: 20:50:15   
2  2023-01-01  Folha Mensal e Complementar Horas: 20:50:15   
3  2023-01-01  Folha Mensal e Complementar Horas: 20:50:15   
4  2023-01-01  Folha Mensal e Complementar Horas: 20:50:15   

                           departamento       vinculo  \
0  2 - ARQDIGITAL ADMINISTRAÇÃO CENTRAL  Contribuinte   
1  2 - ARQDIGITAL ADMINISTRAÇÃO CENTRAL  Contribuinte   
2  2 - ARQDIGITAL ADMINISTRAÇÃO CENTRAL  Contribuinte   
3               3 - ARQDIG